### Create Sqlite Database with the Works Object Column headers

In [1]:
import json
import sqlite3

# Function to extract keys from JSONL file
def extract_keys_from_jsonl(file_path):
    with open(file_path) as f:
        line = f.readline()
        data = json.loads(line)
        return list(data.keys())

# Function to create table in SQLite
def create_table_if_not_exists(table_name, keys):
    conn = sqlite3.connect('sqlite3.db')
    c = conn.cursor()
    
    # Create table if not exists
    c.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join([f'{key} TEXT' for key in keys])})")
    
    conn.commit()
    conn.close()

# Path to your JSONL file
jsonl_file_path = 'part.jsonl'

# Extract keys from JSONL file
keys = extract_keys_from_jsonl(jsonl_file_path)

# Table name
table_name = 'part_sqlite3_table'

# Create table in SQLite
create_table_if_not_exists(table_name, keys)

#### Display the Sqlite headers for confirmation

In [2]:
import sqlite3

# Function to get table columns
def get_table_columns(table_name):
    conn = sqlite3.connect('sqlite3.db')
    c = conn.cursor()
    
    # Query to get table columns
    c.execute(f"PRAGMA table_info({table_name})")
    columns = [row[1] for row in c.fetchall()]
    
    conn.close()
    
    return columns

# Table name
table_name = 'part_sqlite3_table'

# Get table columns
columns = get_table_columns(table_name)
print("Table Columns:", columns)

Table Columns: ['id', 'doi', 'doi_registration_agency', 'display_name', 'title', 'publication_year', 'publication_date', 'language', 'ids', 'primary_location', 'best_oa_location', 'type', 'open_access', 'authorships', 'corresponding_author_ids', 'corresponding_institution_ids', 'cited_by_count', 'summary_stats', 'biblio', 'is_retracted', 'is_paratext', 'concepts', 'mesh', 'locations_count', 'locations', 'referenced_works', 'referenced_works_count', 'sustainable_development_goals', 'grants', 'apc_list', 'apc_paid', 'related_works', 'abstract_inverted_index', 'counts_by_year', 'cited_by_api_url', 'updated_date', 'created_date', 'updated', 'authors_count', 'concepts_count']


### Create Sqlitedict Database with the Works Object Column headers 

In [3]:
from sqlitedict import SqliteDict
import json

# Function to create SqliteDict with dynamic table and column names
def create_sqldict_with_dynamic_schema(file_path, table_name):
    keys = extract_keys_from_jsonl(file_path)

    sqldict = SqliteDict("sqlitedict.sqlite", tablename=table_name, autocommit=True)

    # Insert data into SqliteDict
    with sqldict:
        with open(file_path) as f:
            for line in f:
                data = json.loads(line)
                key = data.get('id')  # Assuming 'id' is unique and used as key
                sqldict[key] = data

    return sqldict

# Path to your JSONL file
jsonl_file_path = 'part.jsonl'

# Table name
table_name = 'sqlitedict_table'

# Create SqliteDict with dynamic schema
sqldict = create_sqldict_with_dynamic_schema(jsonl_file_path, table_name)

# Get table names from the SQLite database
tables = sqldict.get_tablenames('sqlitedict.sqlite')
print(tables)

# Close the SqliteDict
sqldict.close()

['sqlitedict_table']


#### Display the Sqlitedict headers for confirmation

In [ ]:
from sqlitedict import SqliteDict

# Function to get column names (headers) from SqliteDict table
def get_sqldict_columns(sqldict):
    # Get the first item from the SqliteDict
    first_item = next(iter(sqldict.values()), None)
    
    # Extract column names if the dictionary is not empty
    if first_item:
        return list(first_item.keys())
    else:
        return []

# Create a SqliteDict object
sqldict = SqliteDict("sqlitedict.sqlite", tablename="sqlitedict_table", autocommit=True)

# Get column names
columns = get_sqldict_columns(sqldict)
print("Table Columns:", columns)

# Close the SqliteDict
sqldict.close()

#### Compare the sqlite3 and sqlitedict

In [ ]:
import json
import time
import sqlite3
import sqlitedict

# Function to insert data into SQLite
def insert_into_sqlite(data):
    conn = sqlite3.connect('sqlite3.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, data TEXT)''')
    for row in data:
        c.execute('''INSERT INTO test (data) VALUES (?)''', (json.dumps(row),))
    conn.commit()
    conn.close()

# Function to insert data into SqliteDict
def insert_into_sqlitedict(data):
    db = sqlitedict.SqliteDict('sqlitedict.sqlite', autocommit=True)
    for row in data:
        db[str(row['id'])] = row
    db.close()

# Function to read data from SQLite
def read_from_sqlite():
    conn = sqlite3.connect('sqlite3.db')
    c = conn.cursor()
    c.execute('''SELECT * FROM test''')
    rows = c.fetchall()
    conn.close()
    return rows

# Function to read data from SqliteDict
def read_from_sqlitedict():
    db = sqlitedict.SqliteDict('sqlitedict.sqlite', autocommit=True)
    data = [v for k, v in db.items()]
    db.close()
    return data

# Load data from .jsonl file or SQLite database
# Try different sizes
for size_of_db in [10,100,1000, 10000 , 100000]:
    with open(f'part_{size_of_db}.jsonl') as f:
        data = [json.loads(line) for line in f]

    # For demonstration purposes, let's assume 'data' is already loaded with test data
    print("\n...........................................................................\n")
    print(f"\nWorking on the {size_of_db}x data now. Which is {size_of_db * 10} rows.\n")
    # Test Write Performance
    start_time = time.time()
    insert_into_sqlite(data)
    sqlite_write_time = time.time() - start_time

    start_time = time.time()
    insert_into_sqlitedict(data)
    sqlitedict_write_time = time.time() - start_time

    print("SQLite Write Time:", sqlite_write_time)
    print("SqliteDict Write Time:", sqlitedict_write_time)

    # Test Read Performance
    start_time = time.time()
    rows_sqlite = read_from_sqlite()
    sqlite_read_time = time.time() - start_time

    start_time = time.time()
    rows_sqlitedict = read_from_sqlitedict()
    sqlitedict_read_time = time.time() - start_time

    print("SQLite Read Time:", sqlite_read_time)
    print("SqliteDict Read Time:", sqlitedict_read_time)
    print("\n...........................................................................\n")

    # Verify data consistency if needed
    # assert rows_sqlite == rows_sqlitedict


...........................................................................


Working on the 10x data now. Which is 100 rows.

SQLite Write Time: 0.016738176345825195
SqliteDict Write Time: 0.08756017684936523
SQLite Read Time: 18.30529808998108
SqliteDict Read Time: 0.0031557083129882812

...........................................................................


...........................................................................


Working on the 100x data now. Which is 1000 rows.

SQLite Write Time: 0.059914588928222656
SqliteDict Write Time: 0.4994313716888428
SQLite Read Time: 6.604947328567505
SqliteDict Read Time: 0.0029964447021484375

...........................................................................


...........................................................................


Working on the 1000x data now. Which is 10000 rows.

SQLite Write Time: 0.5243668556213379
SqliteDict Write Time: 4.890326023101807
SQLite Read Time: 3.852259397506714
SqliteDict Rea

### Duplicate jsonl content function for Testing purposes only

In [13]:
# # Path to input file (part.jsonl)
# input_file_path = 'part.jsonl'

# # Path to output file (partmany.jsonl)
# output_file_path = 'part_100000.jsonl'

# # Number of times to repeat the content
# repeat_count = 100000

# # Read content from input file
# with open(input_file_path, 'r') as input_file:
#     content = input_file.read()

# # Write content to output file multiple times
# with open(output_file_path, 'w') as output_file:
#     for _ in range(repeat_count):
#         output_file.write(content)